### Author : Antonio Sirica - Course : Computational Intelligence

In [18]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import matplotlib.pyplot as plt 
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [19]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])

#DIST_MATRIX not used in this simple exercise since the shortest path is not asked 
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [20]:
import numpy as np
import networkx as nx

# Initialize an undirected graph
G = nx.Graph() 

# Iterate through each city and find its 5 closest neighbors
for c1 in CITIES.itertuples():
    # Get the unique identifier for the current city (name)
    city_name_1 = c1.name  # Assuming the column is called 'name'
    
    # Get distances from the current city to all other cities
    distances = DIST_MATRIX[c1.Index]  # Row of distances from city c1

    # Find indices of the 5 closest cities (excluding the city itself)
    closest_indices = np.argsort(distances)[:6]  # Top 6 includes the city itself

    # Loop through the closest cities
    for idx in closest_indices[1:]:  # Skip closest_indices[0], which is the city itself
        # Get the full row object for the neighbor city (c2)
        c2 = CITIES.iloc[idx]  # c2 is a Pandas row with the full data
        
        # Get the unique identifier for the neighbor city (name)
        city_name_2 = c2.name  # Assuming the column is called 'name'

        # Add c1 and c2 as nodes using the unique identifier (city_name)
        G.add_node(city_name_1, **c1._asdict())  # Add city c1 with attributes
        G.add_node(city_name_2, **c2.to_dict())  # Add city c2 with attributes

        # Add edge between the two nodes with the distance as weight
        G.add_edge(city_name_1, city_name_2, weight=DIST_MATRIX[c1.Index, idx])

# Check connectivity and print graph info
print(nx.is_connected(G))  # Check if the graph is connected
print(G)  # Print the graph information

#Duplicates of edges are not built


True
Graph with 91 nodes and 230 edges


### Greedy Best First with extra info that is knowing that the goal is in direction north-west

In [22]:

# Heuristic is choosing the "most" north-west decision in this case
def fitness(node_data):
    # Returns a tuple (latitude, longitude) as the fitness
    return (node_data['lat'], -node_data['lon'])

def goal_test(current_node, goal):
    return current_node == goal

# Mapping city names to indices
city_name_to_idx = {row['name']: row.name for _, row in CITIES.iterrows()}

starting_city = 'Naples'
end_city = 'Vicenza'

# Ensure the starting and ending cities exist in the graph
if starting_city not in city_name_to_idx:
    print(f"{starting_city} is not in the cities data.")
    exit()

if end_city not in city_name_to_idx:
    print(f"{end_city} is not in the cities data.")
    exit()

# Get the graph nodes using indices
starting_idx = city_name_to_idx[starting_city]
end_idx = city_name_to_idx[end_city]

# Check if the starting city exists in the graph
if not G.has_node(starting_idx):
    print(f"Node for {starting_city} is not in the graph.")
    exit()

# Check if there is a path from the starting city to the end city
if not nx.has_path(G, starting_idx, end_idx):
    print(f"No path exists between {starting_city} and {end_city}.")
    exit()

# Initialize the starting node's data
node_data = G.nodes[starting_idx]
print("Node attributes for starting city:", node_data)

# Start the loop to explore neighbors
current_node = starting_idx
visited = list()
previous_node = None  # Keep track of the previous node to backtrack

while current_node != end_idx:
    # Mark the current node as visited
    visited.append(current_node)
    
    # Get neighbors of the current node (using indices)
    neighbors = list(G.neighbors(current_node))
    print(f"Neighbors of {current_node}: {neighbors}")
    
    # Initialize variables to track the best neighbor based on fitness
    best_neighbor = None
    best_fitness = (-float('inf'), -float('inf'))  # Start with a very low fitness value
    
    # Evaluate the fitness of each neighbor and choose the best one based on lat and lon
    for neighbor in neighbors:
        if neighbor not in visited:
            neighbor_data = G.nodes[neighbor]
            neighbor_fitness = fitness(neighbor_data)
            print(f"Checking neighbor {neighbor} with fitness {neighbor_fitness}")
            
            # Compare fitness: higher latitude first, then longitude
            if neighbor_fitness > best_fitness:
                best_fitness = neighbor_fitness
                best_neighbor = neighbor
        
    # If we found a best neighbor, move to it
    if best_neighbor:
        print(f"Moving to {best_neighbor}")
        previous_node = current_node  # Save the current node for backtracking
        current_node = best_neighbor
    else:
        # If there are no unvisited neighbors left, backtrack (no valid path)
        print("No unvisited neighbors left. Backtracking...")
        visited.append(current_node)  # Mark the current node as stuck
        current_node = previous_node  # Move back to the previous node
        previous_node = None  # Clear previous node as we're backtracking
    
    # Check if the loop has found the end_city
    if goal_test(current_node, end_idx):
        print(f"Reached {end_city} passing through {len(visited)} cities!")
        break

# Check if we got stuck in a loop without reaching the destination
if current_node != end_idx:
    print("The algorithm couldn't find a path to the destination.")


Node attributes for starting city: {'name': 'Naples', 'lat': 40.85, 'lon': 14.27}
Neighbors of 21: ['Andria', 'Bari', 'Foggia', 'Giugliano in Campania', 'Latina', 'Salerno', 'Taranto']
Checking neighbor Andria with fitness (41.23, -16.29)
Checking neighbor Bari with fitness (41.12, -16.87)
Checking neighbor Foggia with fitness (41.47, -15.55)
Checking neighbor Giugliano in Campania with fitness (40.93, -14.19)
Checking neighbor Latina with fitness (41.47, -12.89)
Checking neighbor Salerno with fitness (40.68, -14.77)
Checking neighbor Taranto with fitness (40.48, -17.240000000000002)
Moving to Latina
Neighbors of Latina: [34, 39, 14, 21, 27]
Checking neighbor 34 with fitness (41.89, -12.5)
Checking neighbor 39 with fitness (42.57, -12.65)
Checking neighbor 14 with fitness (40.93, -14.19)
Checking neighbor 27 with fitness (42.46, -14.21)
Moving to 39
Neighbors of 39: ['Ancona', 'Latina', 'Perugia', 'Pescara', 'Rome', 'Sassari']
Checking neighbor Ancona with fitness (43.6, -13.5)
Checkin